In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [12]:
# Import data
ffdaily = pd.read_csv('ff6_daily_factors.csv')
ffmonthly = pd.read_csv('ff6_monthly_factors.csv')

(25461, 8)

In [3]:
factorlist = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']

### I perform the exercise in Q1 (a)-(f) with just the market factor, you need to do this for all six factors.

In [28]:
factordaily_res = pd.read_csv('ff6_daily_factors.csv', usecols = [0])
for factor in factorlist:
    #factor = 'Mkt-RF'
    factordaily = ffdaily[['date', factor]].copy()
    # Calculate rolling variance
    factordaily['rolling_var_'+factor] = factordaily[factor].rolling(window=22, min_periods=22).var()
    # Calculate factor weight
    targetvol = factordaily['rolling_var_'+factor].mean()
    factordaily['weight_'+factor] = targetvol/factordaily['rolling_var_'+factor]
    # Collapse to Monthly
    factordaily['mdate'] = (factordaily['date']/100).astype(int)
    factordaily = factordaily.drop_duplicates('mdate', keep='last')
    
    factordaily = factordaily[['mdate', 'weight_'+factor, 'rolling_var_'+factor]].copy()
    factordaily = factordaily.rename(columns={'mdate':'date'})
    factordaily_res = pd.merge(factordaily_res, factordaily, on='date', how='right')

# Merge weights
factormonthly = pd.merge(ffmonthly, factordaily_res, on='date', how='left')


In [30]:
factormonthly

,date,Mkt-RF,SMB,HML,RF,MOM,RMW,CMA,weight_Mkt-RF,rolling_var_Mkt-RF,weight_SMB,rolling_var_SMB,weight_HML,rolling_var_HML,weight_RMW,rolling_var_RMW,weight_CMA,rolling_var_CMA,weight_MOM,rolling_var_MOM
0,192607,2.96,-2.56,-2.43,0.22,NaN,NaN,NaN,5.106438,0.228485,2.916002,0.119345,2.331971,0.164283,NaN,NaN,NaN,NaN,NaN,NaN
1,192608,2.64,-1.17,3.82,0.25,NaN,NaN,NaN,3.124637,0.373402,2.705803,0.128616,3.828876,0.100056,NaN,NaN,NaN,NaN,NaN,NaN
2,192609,0.36,-1.40,0.13,0.23,NaN,NaN,NaN,4.579976,0.254749,6.080025,0.057238,6.946363,0.055152,NaN,NaN,NaN,NaN,NaN,NaN
3,192610,-3.24,-0.09,0.70,0.32,NaN,NaN,NaN,1.686221,0.691929,2.213027,0.157255,1.745261,0.219511,NaN,NaN,NaN,NaN,NaN,NaN
4,192611,2.53,-0.10,-0.51,0.31,NaN,NaN,NaN,7.625126,0.153013,7.288544,0.047747,4.565078,0.083921,NaN,NaN,NaN,NaN,3.051566,0.196892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,202211,4.60,-3.40,1.38,0.29,-2.01,6.01,3.11,0.366640,3.182266,0.846766,0.410986,0.312091,1.227538,0.251070,0.616556,0.156994,0.890160,0.118243,5.081281
1157,202212,-6.41,-0.68,1.32,0.33,4.52,0.09,4.19,0.636241,1.833809,1.313233,0.265002,0.462815,0.827769,0.412305,0.375447,0.319449,0.437472,0.347777,1.727624
1158,202301,6.65,5.02,-4.05,0.35,-15.98,-2.62,-4.53,0.998027,1.169051,1.326902,0.262272,0.847666,0.451952,0.522418,0.296312,0.370107,0.377593,0.404714,1.484575
1159,202302,-2.58,1.21,-0.78,0.34,0.20,0.90,-1.41,1.044963,1.116542,0.658482,0.528502,0.428205,0.894674,0.582316,0.265832,0.240643,0.580736,0.221154,2.716785


In [43]:
# Is volatility persistent?
for factor in factorlist:
    voldf = factormonthly[['date', 'rolling_var_'+factor, factor]].copy()
    voldf['lag_vol'] = voldf['rolling_var_'+factor].shift(1)
    voldf = voldf.dropna()
    modelvol=sm.OLS(voldf['rolling_var_'+factor],
            sm.add_constant(voldf[['lag_vol']])).fit()
    print(modelvol.summary())
    # Does Vol predict factor returns?
    modelvol=sm.OLS(voldf[factor],
            sm.add_constant(voldf[['lag_vol']])).fit()
    print(modelvol.summary())

                            OLS Regression Results                            
Dep. Variable:     rolling_var_Mkt-RF   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     466.4
Date:                Sat, 23 Sep 2023   Prob (F-statistic):           3.39e-87
Time:                        21:32:05   Log-Likelihood:                -2535.5
No. Observations:                1160   AIC:                             5075.
Df Residuals:                    1158   BIC:                             5085.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5418      0.070      7.790      0.0

In [39]:
for factor in factorlist:
    factormonthly['Ret_'+factor] = factormonthly['weight_'+factor].shift(1)*(factormonthly[factor])
factormonthly

,date,Mkt-RF,SMB,HML,RF,MOM,RMW,CMA,weight_Mkt-RF,rolling_var_Mkt-RF,...,weight_CMA,rolling_var_CMA,weight_MOM,rolling_var_MOM,Ret_MOM,Ret_Mkt-RF,Ret_SMB,Ret_HML,Ret_RMW,Ret_CMA
0,192607,2.96,-2.56,-2.43,0.22,NaN,NaN,NaN,5.106438,0.228485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,192608,2.64,-1.17,3.82,0.25,NaN,NaN,NaN,3.124637,0.373402,...,NaN,NaN,NaN,NaN,NaN,13.480997,-3.411722,8.908129,NaN,NaN
2,192609,0.36,-1.40,0.13,0.23,NaN,NaN,NaN,4.579976,0.254749,...,NaN,NaN,NaN,NaN,NaN,1.124869,-3.788124,0.497754,NaN,NaN
3,192610,-3.24,-0.09,0.70,0.32,NaN,NaN,NaN,1.686221,0.691929,...,NaN,NaN,NaN,NaN,NaN,-14.839122,-0.547202,4.862454,NaN,NaN
4,192611,2.53,-0.10,-0.51,0.31,NaN,NaN,NaN,7.625126,0.153013,...,NaN,NaN,3.051566,0.196892,NaN,4.266138,-0.221303,-0.890083,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,202211,4.60,-3.40,1.38,0.29,-2.01,6.01,3.11,0.366640,3.182266,...,0.156994,0.890160,0.118243,5.081281,-0.773922,1.700959,-3.206134,0.778701,1.561273,0.912489
1157,202212,-6.41,-0.68,1.32,0.33,4.52,0.09,4.19,0.636241,1.833809,...,0.319449,0.437472,0.347777,1.727624,0.534460,-2.350160,-0.575801,0.411961,0.022596,0.657806
1158,202301,6.65,5.02,-4.05,0.35,-15.98,-2.62,-4.53,0.998027,1.169051,...,0.370107,0.377593,0.404714,1.484575,-5.557481,4.231005,6.592430,-1.874401,-1.080239,-1.447104
1159,202302,-2.58,1.21,-0.78,0.34,0.20,0.90,-1.41,1.044963,1.116542,...,0.240643,0.580736,0.221154,2.716785,0.080943,-2.574910,1.605551,-0.661180,0.470176,-0.521851


In [45]:
# Asset pricing test of the CAPM
for factor in factorlist:
    cols = ['Ret_'+factor, factor]
    returndf = factormonthly[cols].copy().dropna()
    model1=sm.OLS(returndf['Ret_'+factor],
            sm.add_constant(returndf[[factor]])).fit()
    print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             Ret_Mkt-RF   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     675.1
Date:                Sat, 23 Sep 2023   Prob (F-statistic):          1.25e-117
Time:                        21:34:09   Log-Likelihood:                -4464.1
No. Observations:                1160   AIC:                             8932.
Df Residuals:                    1158   BIC:                             8942.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9393      0.336      2.794      0.0

In [46]:
# AP Test FF3
for factor in factorlist:
    cols = ['Ret_'+factor, 'Mkt-RF','SMB', 'HML']
    returndf = factormonthly[cols].copy().dropna()
    model2=sm.OLS(returndf['Ret_'+factor],
        sm.add_constant(returndf[['Mkt-RF','SMB', 'HML']])).fit()
    print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:             Ret_Mkt-RF   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     233.8
Date:                Sat, 23 Sep 2023   Prob (F-statistic):          1.56e-118
Time:                        21:35:20   Log-Likelihood:                -4455.4
No. Observations:                1160   AIC:                             8919.
Df Residuals:                    1156   BIC:                             8939.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0412      0.335      3.109      0.0

In [47]:
# AP Test FF5 + MOM
for factor in factorlist:
    cols = ['Ret_'+factor, 'Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']
    returndf = factormonthly[cols].copy().dropna()
    model3=sm.OLS(returndf['Ret_'+factor],
        sm.add_constant(returndf[['Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']])).fit()
    print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:             Ret_Mkt-RF   R-squared:                       0.467
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     103.8
Date:                Sat, 23 Sep 2023   Prob (F-statistic):           1.17e-93
Time:                        21:36:15   Log-Likelihood:                -2604.9
No. Observations:                 717   AIC:                             5224.
Df Residuals:                     710   BIC:                             5256.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0961      0.365      0.264      0.7